**Imports and global declarations**

In [1]:
# Hoang-Nam Tran, z5629534

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score, precision_score, mean_absolute_error
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import r_regression
import csv


# Data columns before pre-processing
# ['year', 'month', 'u10', 'v10', 'mx2t', 'mn2t', 'tcc', 't2', 'msl', 't', 'q', 'u', 'v', 'z', 'SPI', 'grid_ID']

# Data columns after pre-processing
# ['year', 'month', 'u10', 'v10', 'mx2t', 'mn2t', 'tcc', 't2', 'msl', 't', 'q', 'u', 'v', 'z', 'SPI', 'grid_ID','Drought', 'monthCos', 'monthSin']
# monthCos and monthSin are the cosine and sine of the normalised month, respectively

#used to determine and print the used predictors
allPredictors = ['year', 'month', 'u10', 'v10', 'mx2t', 'mn2t', 'tcc', 't2', 'msl', 't', 'q', 'u', 'v', 'z', 'SPI', 'grid_ID', 'Drought', 'monthCos', 'monthSin']

# forbidden indices: year, SPI, grid_ID, Drought -> 0, 14, 15, 16
forbiddenColumns = [0, 14, 15, 16]

#seed for reproducibility
shuffleSeed = 42

#seeding keras for reproducibility of training
keras.utils.set_random_seed(42)


**Method declarations for pre-processing**

In [2]:
# Initializing drought variable based on SPI (index 14)
def addDrought(data):
    for x in data:
        try:
            if float(x[14]) <= -1:
                x.append(1)
            else:
                x.append(0)
        except:
            #placeholder 'Drought' if no calculation possible
            x.append('Drought')
    return data


#removing rows with non-numerical values
def checkNonFloatNonNP(data):
    for x in data:
        for y in x:
            try:
                float(y)
            except:
                data.remove(x)
                break


#removing rows containing invalid months
def filterInvalidMonthsNP(data):
    new_data = []
    validMonths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    for x in data:
        if x[1] in validMonths:
            new_data.append(x)
    return np.array(new_data)


#normalising months, cyclic encoding with sin and cos
def normaliseMonth(dataSet):
    new_dataSet = []
    for x in dataSet:
        month = x[1]
        month_normalised = 2 * np.pi * (month - 1) / 12
        x = np.append(x, [np.cos(month_normalised), np.sin(month_normalised)])
        new_dataSet.append(x)
    return np.array(new_dataSet)


#removing rows with infinite values
def noIncludeInfinites(data):
    new_data = []
    detected = False
    for x in data:
        for y in x:
            if y == float('inf') or y == float('-inf'):
                detected = True
                break
        if detected == False:
            new_data.append(x)
    return np.array(new_data)


#detecting outliers (column-wise) with z-score method, threshold = 3
def detectOutliersByColumn(data, column):
    mean = np.mean(data[:, column])
    std_dev = np.std(data[:, column])
    z_scores = (data[:, column] - mean) / std_dev
    outliers = np.abs(z_scores) > 3
    outlier_indices = np.where(outliers)
    return outlier_indices


#removing rows with outliers
def removeOutliers(data, excludedColumns):
    outliersIndices = set()
    #iterating through all columns
    for i in range(0, len(data[0])):
        #excluding the forbidden columns
        if i not in excludedColumns:
            outlierIndicesColumn = detectOutliersByColumn(data, i)
            for x in outlierIndicesColumn[0]:
                outliersIndices.add(x)

    outliersIndices = list(outliersIndices)
    outliersIndices.sort()
    data = np.delete(data, outliersIndices, axis=0)
    return data

**Method declarations for evaluation**

In [3]:
#3f, Create a plot showing the accuracy (y-axis) versus the number of epochs (x-axis) for both the training and validation sets.
def plotAccuracy(result):
    plt.plot(result.history['accuracy'], 'b', label='Training accuracy')
    plt.plot(result.history['val_accuracy'], 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


#4e, Creating a plot showing the loss value (y-axis) versus the number of epochs (x-axis) for both the training and validation sets.
def plotLoss(result):
    plt.plot(result.history['loss'], 'b', label='Training loss')
    plt.plot(result.history['val_loss'], 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


#Create a scatter plot showing predicted SPI (y-axis) versus true SPI (x-axis)
def scatterPlot(target, predicted):
    plt.scatter(target, predicted)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('SPI vs Predicted SPI')
    plt.show()


#Compute and plot a confusion matrix. Positive class is 1, i.e. ‘Drought’.
def plotSimpleConfusionMatrix(target, predicted):
    confusion_matrix = metrics.confusion_matrix(target, predicted)
    cmDisplay = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["No Drought", "Drought"])
    cmDisplay.plot()
    plt.show()

#Printing used predictors, selecting the predictors from allVariables based on the given indices
def printPredictorsSet(allVariables, indices):
    usedPredictors = []
    for x in indices:
        usedPredictors.append(allVariables[x])
    print("Used predictors: ", usedPredictors)

**Calculating drought from SPI and pre-processing of data – 3a, 3c, 4b**

In [4]:
#uncomment from here to skip training

with open('Climate_SPI.csv', newline='') as csvfileUnseen:
    initData = list(csv.reader(csvfileUnseen))

#pre-processing, adding drought variable, removing invalid rows
initData = addDrought(initData)
checkNonFloatNonNP(initData)

#shuffling the data for randomness in training
random.Random(shuffleSeed).shuffle(initData)

initData = np.array(initData)
initData = initData.astype(float)


#removing outliers (z-score, threshold=3), invalid months, infinite values, normalising months
initData = removeOutliers(initData, forbiddenColumns)
initData = filterInvalidMonthsNP(initData)
initData = noIncludeInfinites(initData)
initData = normaliseMonth(initData)


#predictors used for training and prediction
#same predictors for classification and regression since using different predictors did not show improvement
inputColumns = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17, 18]

#3h, 4g, testing different subsets of predictors
#in the end, using all predictors was the best option
# inputColumns = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18]
# inputColumns = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18]
# inputColumns = [2, 3, 4, 5, 6, 7, 9, 10, 11, 12]
# inputColumns = [4, 6, 7, 8, 10, 11, 12, 13], test removing related predictors


#selecting correct columns for input data
unseenInput = initData[:, inputColumns]


#normalising the input data with min-max normalisation
scaler = MinMaxScaler()
scaler.fit(unseenInput)
unseenInputNormalised = scaler.transform(unseenInput)


#selecting target-columns for classification and regression, Drought (index 16) and SPI (index 14)
unseenTargetClassification = initData[:, 16]
unseenTargetRegression = initData[:, 14]


**Splitting data into training (70%), validation (15%) and test sets (15%) – 3b, 4a**

In [5]:
#Data was shuffled before, randomness ensured

#Split data by selecting correct rows for training, validation and testing
#Split happens after pre-processing, so same transformation is applied to all sets
inputTrain = unseenInputNormalised[:int(0.7*len(initData))]
inputVal = unseenInputNormalised[int(0.7*len(initData)):int(0.85*len(initData))]
inputTest = unseenInputNormalised[int(0.85*len(initData)):]


targetTrainClass = unseenTargetClassification[:int(0.7*len(initData))]
targetValClass = unseenTargetClassification[int(0.7*len(initData)):int(0.85*len(initData))]
targetTestClass = unseenTargetClassification[int(0.85*len(initData)):]


targetTrainRegression = unseenTargetRegression[:int(0.7*len(initData))]
targetValRegression = unseenTargetRegression[int(0.7*len(initData)):int(0.85*len(initData))]
targetTestRegression = unseenTargetRegression[int(0.85*len(initData)):]

**Building and training the classification model – 3d, 3e**

**Plot of the accuracy (y-axis) versus the number of epochs (x-axis) for both the training and validation sets – 3f**

In [ ]:
classificationModel = Sequential()
#hidden layer with 80 neurons
classificationModel.add(Dense(80, activation='relu', input_dim=14))
classificationModel.add(Dense(1, activation='sigmoid'))

adam = keras.optimizers.Adam(learning_rate=0.001)
classificationModel.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
resultClass = classificationModel.fit(inputTrain, targetTrainClass, epochs=200, batch_size=64, validation_data=(inputVal, targetValClass))

plotAccuracy(resultClass)

**Building and training the regression model – 4c, 4d**

**Plot showing the loss value (y-axis) versus the number of epochs (x-axis) for both the training and validation sets – 4e**

In [ ]:
regressionModel = Sequential()
#hidden layer with 80 neurons
regressionModel.add(Dense(80, activation="relu", input_dim=14))
regressionModel.add(Dense(1))

adamReg = keras.optimizers.Adam(learning_rate=0.001)
regressionModel.compile(loss='mean_squared_error', optimizer=adamReg, metrics=["mean_squared_error"])
resultRegression = regressionModel.fit(inputTrain, targetTrainRegression, epochs=100, batch_size=64, validation_data=(inputVal, targetValRegression))

plotLoss(resultRegression)

**Evaluating classification model on test set**

**Confusion matrix, performance metrics “Balanced Accuracy” and “Precision” calculated on the test set**

In [ ]:
#predicting drought on the test set, binary classification with threshold 0.5, 3j
predictedClass = classificationModel.predict(inputTest)
predictedClassBinary = (predictedClass >= 0.5).astype("int32")


#plotting the confusion matrix and performance metrics, 3k
plotSimpleConfusionMatrix(targetTestClass, predictedClassBinary)
print("Balanced Accuracy: ", balanced_accuracy_score(targetTestClass, predictedClassBinary))
print("Precision: ", precision_score(targetTestClass, predictedClassBinary, pos_label=1))


**Evaluating regression model on test set**

**On the test set: scatter plot, “Mean Absolute Error (MAE)” and the “Pearson Correlation
Coefficient” between the true and predicted SPI**

In [ ]:
#predicting SPI on the test set, 4i
predictedRegression = regressionModel.predict(inputTest)

#scatter plot and performance metrics, 4j
scatterPlot(targetTestRegression, predictedRegression)
print("Mean Absolute Error: ", mean_absolute_error(targetTestRegression, predictedRegression))
print("Pearson Correlation Coefficient: ", r_regression(predictedRegression, targetTestRegression))

**Saving models and loading saved models for task 5**

In [10]:
classificationModel.save("classificationDemo.keras")
regressionModel.save("regressionDemo.keras")

#uncomment until here to skip training

classificationModelLoaded = keras.models.load_model("classificationDemo.keras") #5.1d
regressionModelLoaded = keras.models.load_model("regressionDemo.keras") #5.2c

**Pre-processing unseen data**

In [11]:
# might need to change file name during discussion, 5a
with open('Fake_Climate_SPI6.csv', newline='') as csvfileUnseen:
    unseenData = list(csv.reader(csvfileUnseen))

#pre-processing and normalisation of the unseen data with the same steps as on the previous data, 5.1c, 5.2b
#Adding drought variable, removing invalid rows
unseenData = addDrought(unseenData) #5.1b
checkNonFloatNonNP(unseenData)

unseenData = np.array(unseenData)
unseenData = unseenData.astype(float)

#removing outliers (z-score, threshold=3), invalid months, infinite values, normalising months
unseenData = removeOutliers(unseenData, forbiddenColumns)
unseenData = filterInvalidMonthsNP(unseenData)
unseenData = noIncludeInfinites(unseenData)
unseenData = normaliseMonth(unseenData)


#selecting correct columns for input and target data, drought (index 16) and SPI (index 14)
unseenInput = unseenData[:, inputColumns]
unseenTargetClassification = unseenData[:, 16]
unseenTargetRegression = unseenData[:, 14]


#normalising the unseen input data with min-max normalisation
scalerUnseen = MinMaxScaler()
scalerUnseen.fit(unseenInput)
unseenInputNormalised = scalerUnseen.transform(unseenInput)

**Predictions and evaluations on unseen data**

In [ ]:
#predicting drought, binary classification on unseen data, threshold 0.5, 5.1e
unseenClassPred = classificationModelLoaded.predict(unseenInputNormalised)
unseenPredBin = (unseenClassPred >= 0.5).astype("int32")


#plotting the confusion matrix and performance metrics
print("Classification on unseen data:")
plotSimpleConfusionMatrix(unseenTargetClassification, unseenPredBin) #5.1f

#5.1g
print("Balanced Accuracy: ", balanced_accuracy_score(unseenTargetClassification, unseenPredBin))
print("Precision: ", precision_score(unseenTargetClassification, unseenPredBin, pos_label=1))
#5.1h
print("Number of samples: ", len(unseenData))
printPredictorsSet(allPredictors, inputColumns)

print("\n")

#predicting SPI on the unseen data, 5.2d
unseenRegressionPred = regressionModelLoaded.predict(unseenInputNormalised)


#scatter plot and performance metrics
print("Regression on unseen data:")
scatterPlot(unseenTargetRegression, unseenRegressionPred) #5.2e
#5.2f
print("Mean Absolute Error: ", mean_absolute_error(unseenTargetRegression, unseenRegressionPred))
print("Pearson Correlation Coefficient: ", r_regression(unseenRegressionPred, unseenTargetRegression))
#5.2g
print("Number of samples: ", len(unseenData))
printPredictorsSet(allPredictors, inputColumns)
